In [1]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.storage import InMemoryByteStore
from langchain.retrievers import ParentDocumentRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.output_parsers import StrOutputParser

In [3]:
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [4]:
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  
    max_tokens=200,             
)

In [5]:
pdf_path = "os-sertoes.pdf"
load_pdf = PyPDFLoader(pdf_path, extract_images=False)
pages = load_pdf.load_and_split()

In [6]:
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200  
)

parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,          
    chunk_overlap=200,        
    length_function=len,      
    add_start_index=True      
)

In [12]:
store = InMemoryByteStore()
vectorstore = Chroma(embedding_function=embeddings_model)

In [8]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,       
    docstore=store,                
    child_splitter=child_splitter, 
    parent_splitter=parent_splitter 
)

parent_document_retriever.add_documents(pages, ids=None)

In [9]:
TEMPLATE = """
Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado:

Contexto: {context}

Pergunta: {question}
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=TEMPLATE)

parser = StrOutputParser()

sequence = RunnableSequence(prompt | llm | parser)

In [10]:
def answer_question(question: str):
    context = parent_document_retriever.invoke(question)

    response = sequence.invoke({"context": context, "question": question})

    return response

In [11]:
for index, question in enumerate(questions):
    resposta = answer_question(question)
    print({"numero": index, "pergunta": question, "resposta": resposta})

{'numero': 0, 'pergunta': 'Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?', 'resposta': 'Euclides da Cunha descreve o ambiente natural do sertão nordestino como um local extremamente árido e desafiador, marcado pela seca e pela escassez de recursos hídricos. Ele destaca a influência da seca na vida dos habitantes, descrevendo o martírio do homem como reflexo de uma tortura maior, que abrange a economia geral da vida. Euclides da Cunha aponta a seca como a principal dificuldade enfrentada pelos habitantes do sertão, destacando a importância da água e a luta constante contra o deserto. Ele descreve a paisagem árida e impiedosa do sertão como um elemento que molda a vida dos habitantes, tornando a sobrevivência uma verdadeira luta diária.'}
{'numero': 1, 'pergunta': 'Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o am